# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [64]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [65]:
cities = pd.read_csv("../output_data/cities.csv").drop(['Unnamed: 0'],axis=1)
cities.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,concepcion del oro,24.63,-101.42,53.71,45,0,6.82,MX,1604282335
1,douglas,41.30,-96.17,39.99,51,1,4.54,US,1604282335
2,nevers,46.92,3.33,63.00,83,100,4.83,FR,1604282336
3,nokaneng,-19.67,22.27,71.20,36,17,5.88,BW,1604282336
4,nova olimpia,-14.80,-57.29,75.29,66,93,1.97,BR,1604282086


In [66]:
cities.dtypes


City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [68]:
#Set up the layout  of the map
fig = gmaps.figure(center=(20,-0.75), zoom_level=2.2)

In [69]:
coordinates = cities[["Lat", "Lng"]]

In [70]:
# Plot Heatmap
#fig = gmaps.figure()


# Mock up some rating data:
humidity = cities["Humidity"]
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=80,
                                 point_radius=1.5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [71]:
my_vacation_cities = cities[(cities["Humidity"]<=30)&(cities["Max Temp"] < 75.00)
                            &(cities["Cloudiness"]<=50.00)&(cities["Max Temp"] >40)]
my_vacation_cities = my_vacation_cities.reset_index(drop=True)
my_vacation_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,gazanjyk,39.24,55.52,64.29,28,35,8.23,TM,1604282338
1,hangu,39.25,117.79,53.60,18,0,20.13,CN,1604282346
2,saint george,37.10,-113.58,71.60,11,1,3.36,US,1604282114
3,arys,42.43,68.80,41.00,30,0,8.95,KZ,1604282362
4,ridgecrest,35.62,-117.67,69.01,10,43,3.00,US,1604282363
5,calama,-22.47,-68.93,55.40,24,0,9.17,CL,1604282390
6,batesville,35.77,-91.64,51.80,29,1,7.63,US,1604282416
7,kant,27.80,79.80,64.27,29,0,5.14,IN,1604282418
8,san ramon,37.78,-121.98,73.40,25,1,0.81,US,1604282404
9,rajgarh,28.63,75.38,67.46,24,0,8.25,IN,1604282232


In [72]:
#df_"Hotel_Name" = []
#like restaurant activity
#hotel is actually lodging
#temp < 75 and > 55, wind < 10,  humidity < 40 and cloudiness < 40
# create a params dict that will be updated with new city each iteration
params = {"key": g_key,
         "radius": 5000,
          "types": "lodging",
         "keyword": "Hotel"}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in my_vacation_cities.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']

    # update address key value
    params["location"] = f"{lat},{lng}"

    # make request
    hotel_name = requests.get(base_url, params=params).json()
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    #cities_lat_lng = cities_lat_lng.json()

    try:
        my_vacation_cities.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        my_vacation_cities.loc[index, "Hotel Name"] = "No Hotel Matched"

# Visualize to confirm lat lng appear
my_vacation_cities


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,gazanjyk,39.24,55.52,64.29,28,35,8.23,TM,1604282338,Railway motel
1,hangu,39.25,117.79,53.60,18,0,20.13,CN,1604282346,Hanting Hotel
2,saint george,37.10,-113.58,71.60,11,1,3.36,US,1604282114,Tru By Hilton St. George
3,arys,42.43,68.80,41.00,30,0,8.95,KZ,1604282362,"Gostinitsa ""Madani"""
4,ridgecrest,35.62,-117.67,69.01,10,43,3.00,US,1604282363,SpringHill Suites by Marriott Ridgecrest
5,calama,-22.47,-68.93,55.40,24,0,9.17,CL,1604282390,Park Hotel Calama
6,batesville,35.77,-91.64,51.80,29,1,7.63,US,1604282416,Hampton Inn Batesville
7,kant,27.80,79.80,64.27,29,0,5.14,IN,1604282418,No Hotel Matched
8,san ramon,37.78,-121.98,73.40,25,1,0.81,US,1604282404,San Ramon Marriott
9,rajgarh,28.63,75.38,67.46,24,0,8.25,IN,1604282232,Hotel Gulab Palace


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
hotel_df = my_vacation_cities
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,gazanjyk,39.24,55.52,64.29,28,35,8.23,TM,1604282338,Railway motel
1,hangu,39.25,117.79,53.60,18,0,20.13,CN,1604282346,Hanting Hotel
2,saint george,37.10,-113.58,71.60,11,1,3.36,US,1604282114,Tru By Hilton St. George
3,arys,42.43,68.80,41.00,30,0,8.95,KZ,1604282362,"Gostinitsa ""Madani"""
4,ridgecrest,35.62,-117.67,69.01,10,43,3.00,US,1604282363,SpringHill Suites by Marriott Ridgecrest
5,calama,-22.47,-68.93,55.40,24,0,9.17,CL,1604282390,Park Hotel Calama
6,batesville,35.77,-91.64,51.80,29,1,7.63,US,1604282416,Hampton Inn Batesville
7,kant,27.80,79.80,64.27,29,0,5.14,IN,1604282418,No Hotel Matched
8,san ramon,37.78,-121.98,73.40,25,1,0.81,US,1604282404,San Ramon Marriott
9,rajgarh,28.63,75.38,67.46,24,0,8.25,IN,1604282232,Hotel Gulab Palace


In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [82]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
fig = gmaps.figure(center=(20, -0.75), zoom_level=2)

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))